<a href="https://colab.research.google.com/github/yesoly/2019_cau_oss_hackathon/blob/master/signature_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import re
import glob
from PIL import Image
import os, re
import tensorflow.compat.v1 as tf

In [0]:
# 파일 경로 지정하기
images = glob.glob('/Level1/*.bmp') # 파일 경로

# 전처리

파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문 > https://wikibook.co.kr/python-machine-learning/ [예제 코드:ch7/avhash.py 참고]

---



In [0]:
def average_hash(fname, size = 100):
    img = Image.open(fname) # 이미지 데이터 열기
    img = img.convert('L') # 그레이스케일로 변환하기
    img = img.resize((size, size), Image.ANTIALIAS) # 56 * 28 리사이즈
    pixel_data = img.getdata() # 픽셀 데이터 가져오기
    pixels = np.array(pixel_data) # Numpy 배열로 변환하기
    pixels = pixels.reshape((size, size)) # 2차원 배열로 변환하기
    avg = pixels.mean() # 평균 구하기
    diff = 1 * (pixels > avg) # 평균보다 크면 1, 작으면 0으로 변환하기
    return diff
    
# 이진 해시로 변환하기
def np2hash(ahash):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 = "".join(sl)
        i = int(s2, 2) # 이진수를 정수로 변환하기
        bhash.append("%04x" % i)
    return "".join(bhash)

# 로지스틱 회귀로 학습

In [0]:
# 학습을 위한 라벨 값 설정 및 해시코드 설정
class User:
  def __init__(self, num, label, name, level,b_hash):
    self.num = num
    self.label = label
    self.name = name
    self.level = level
    self.b_hash = b_hash

img_hash_list = []
img_label_list = []

for fname in images:
  bhash = average_hash(fname)
  img_hash_list.append(bhash.reshape(10000))
  #print(np2hash(ahash))

  label_temp = fname.split('L')[2]
  num = int(label_temp.split('-')[0])

  if (num < 160): # Level1
    user = User(num, 0.0, '나영수', 1, bhash)
    img_label_list.append(0)
  elif (num > 160 and num < 200):
    user = User(num, 1.0, '윤경식', 1, bhash)
    img_label_list.append(1)
  elif (num > 200 and num < 240):
    user = User(num, 0.0, '이서현', 2, bhash)
    img_label_list.append(0) 
  elif(num > 240 and num < 300):
    user = User(num, 1.0, '임민정', 2, bhash)
    img_label_list.append(1)
  elif(num > 300 and num < 320):
    user = User(num, 0.0, '정소정', 3, bhash)
    img_label_list.append(0)
  elif(num > 320 and num < 340):
    user = User(num, 1.0, '박예솔', 3, bhash)
    img_label_list.append(1)

In [0]:
size_row    = 100    # height of the image
size_col    = 100    # width of the image

data = img_hash_list
num_image = len(data)
print(num_image)

for i in range (len(data)) :
  plt.imshow(data[i].reshape(size_row, size_col), cmap='Greys', interpolation='None')
  plt.show()

데이터 학습

In [0]:
X = tf.placeholder(tf.float32, shape=[None, 10000])
Y = tf.placeholder(tf.float32, shape=[num_image])

W = tf.Variable(tf.random_normal([10000, 1], mean=0.01, stddev=0.01), name='weight')
b =  tf.Variable(tf.random_normal([1]), name='bias')
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train =  tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(cost)

predicted =  tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:     
    sess.run(tf.global_variables_initializer())
    feed = {X: img_hash_list, Y: img_label_list}
    for step in range(10001):
        sess.run(train, feed_dict=feed)
        if step % 2000 == 0:
            print(step, sess.run(cost, feed_dict=feed))
       
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict=feed)
    print("\nHypothesis: ", h,"\nCorrect (Y): ", c, "\nAccuracy: ", a)